In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
vessel_name = 'OOCL_GENOA'

# 전항지 출항 확인

전항지 출항 지점 확인

In [3]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [4]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

# 시각화 툴로 processed된 것들 확인

항로 클러스터링

In [5]:
# dense_upper_china_list = ['COSCO_FOS_VRKR6_23processed'
# ,'AKA_BHUM_9V7282_7processed'
# , 'COSCO_FOS_VRKR6_25processed'
# , 'MSC_LA_SPEZIA_DIGX2_2processed'
# , 'OOCL_NEW_YORK_VRJS2_2processed'
# , 'MSC_WASHINGTON_5LDP4_4processed'
# , 'MSC_SOFIA_PAZ_D5FL9_2processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_26processed'
# , 'OOCL_HAMBURG_VRZK9_6processed'
# , 'SEAMAX_STRATFORD_V7EP2_6processed'
# , 'OOCL_LUXEMBOURG_VRGO3_5processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_25processed'
# , 'MSC_LUCIANA_3FTY3_4processed'
# , 'MSC_RENEE_VRPL2_5processed'
# , 'COSCO_FOS_VRKR6_24processed'
# , 'COSCO_FOS_VRKR6_20processed'
# , 'COSCO_FOS_VRKR6_26processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_23processed'
# , 'OOCL_HAMBURG_VRZK9_5processed']

# OOCL_GENOA_voyages_list = [
#     'COSCO_FOS_VRKR6_2022_20processed', 'COSCO_FOS_VRKR6_2022_23processed',
#     'COSCO_FOS_VRKR6_2022_24processed', 'COSCO_FOS_VRKR6_2022_25processed',
#     'AKA_BHUM_9V7282_2022_7processed', 'COSCO_FOS_VRKR6_2022_27processed',
#     'COSCO_FOS_VRKR6_2022_28processed', 'AKA_BHUM_9V7282_2022_7processed',
#     'COSCO_FOS_VRKR6_2022_30processed'
# ]

# OOCL_GENOA_voyages_list = [
#     'AKA_BHUM_9V7282_2022_7processed', 'AKA_BHUM_9V7282_2022_8processed'
# ]
# OOCL_GENOA_voyages_list = [
#     'OOCL_LUXEMBOURG_VRGO3_2022_6processed',
#     'OOCL_NEW_YORK_VRJS2_2022_2processed',
#     'OOCL_NEW_YORK_VRJS2_2022_3processed'
# ]

# OOCL_GENOA_voyages_list = [
#     'OOCL_LUXEMBOURG_VRGO3_2022_5processed',
#     'OOCL_LUXEMBOURG_VRGO3_2022_6processed'
# ]

OOCL_GENOA_voyages_list = [
    'OOCL_GENOA_VROK9_2022_3processed', 'OOCL_GENOA_VROK9_2022_5processed'
]

OOCL_HAMBURG_voyages_list = [
    'OOCL_HAMBURG_VRZK9_2022_5processed', 'OOCL_HAMBURG_VRZK9_2022_6processed'
]

SEAMAX_STRATFORD_voyages_list = [
    'SEAMAX_STRATFORD_V7EP2_2022_6processed',
    'SEAMAX_STRATFORD_V7EP2_2022_7processed'
]

ZIM_CHARLESTORN_voyages_list = [
    'ZIM_CHARLESTON_VRUC3_2022_6processed',
    'ZIM_CHARLESTON_VRUC3_2022_7processed'
]

single_voyage_list = [
    'MSC_ARIANE_3FDA3_2022_1processed', 'MSC_DANIT_3FZU8_2022_2processed',
    'MSC_EVA_3EVM7_2022_1processed', 'MSC_EXPRESS_III_A8LG7_2022_1processed',
    'MSC_KANOKO_D5TM3_2023_1processed', 'MSC_LA_SPEZIA_DIGX2_2022_2processed',
    'MSC_VANESSA_HOWD_2023_1processed', 'MSC_VIRGO_CQEB9_2023_2processed',
    'MSC_SOFIA_PAZ_D5FL9_2022_2processed',
    'MSC_WASHINGTON_5LDP4_2022_4processed',
    'MSC_RAVENNA_A8ZU8_2022_3processed', 'MSC_VANDYA_3FDW4_2022_2processed',
    'MSC_RENEE_VRPL2_2022_5processed', 'SEASPAN_OCEANIA_VRBI2_2023_1processed',
    'SEASPAN_BREEZE_VRNL8_2023_1processed', 'VALENCE_9HA3398_2023_1processed'
]

In [6]:
upper_china_path_dir = 'voy_path/upper_china/processed/'

In [7]:
# 가는 길인데 굳이 오래 돌아갈 필요는 없잖아 알고리즘
def optimal_next_point(current_position, shortest_point_idx, shortest_distance,
                       path_df):
    posi_c = current_position
    latitude_i = path_df.loc[shortest_point_idx, "latitude"]
    longitude_i = path_df.loc[shortest_point_idx, "longitude"]
    posi_i = (latitude_i, longitude_i)
    min_d = shortest_distance
    n = shortest_point_idx + 1
    while True:
        try:
            latitude_n = path_df.loc[n, "latitude"]
            longitude_n = path_df.loc[n, "longitude"]
            posi_n = (latitude_n, longitude_n)
            distance = haversine(posi_c, posi_n, unit="km")
            if n == shortest_point_idx + 1:
                path_distance = haversine(posi_i, posi_n, unit="km")
                paths_distance = path_distance
            else:
                path_distance = haversine(posi_n, posi_previous_n, unit="km")
                paths_distance += path_distance

            if distance < (paths_distance + min_d):
                optimal_next_point_idx = n
                return optimal_next_point_idx
            elif n == path_df.index[-1]:
                print("no better optimal_point")
                return shortest_point_idx + 1
            else:
                n = n + 1
                posi_previous_n = posi_n
        except:
            print("no better optimal_point")
            return shortest_point_idx + 1

In [8]:
# 인덱스 번호를 가지고 항로의 segment 번호를 찾음
# 만약 segment의 마지막 번호라면 seg_no를 하나 더한다.

def find_segment_no(optimal_point_idx, path_df, the_number_of_segment):
    seg_no = optimal_point_idx // (len(path_df) / the_number_of_segment)
    next_no = (optimal_point_idx + 1) // (len(path_df) / the_number_of_segment)
    if seg_no != next_no:
        return int(next_no)
    else:
        return int(seg_no)

In [9]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [10]:
def cal_duration(speed, distance):
    if speed == 0:
        speed = 0.1
    hour = int(distance // speed)
    minute = int(
        (distance / speed - hour) * 60)
    second = ((distance / speed - hour) * 60 - minute) * 60
    duration = datetime.timedelta(hours=hour,minutes=minute, seconds=second)
    return duration

# Tianjin to Busan PNIT

## OOCL_LUXEMBOURG

In [11]:
coordinates_array = pd.DataFrame(data={"latitude" : [],"longitude" : []})

for i in OOCL_GENOA_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    coordinates_array = coordinates_array.append(df[["latitude","longitude"]], ignore_index=True)

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from shapely.geometry import LineString

In [14]:
from scipy import interpolate
import pandas as pd

## below 3 knot 제거

In [15]:
interpolation_number = "12000"

### interpolation 2000

In [16]:
max_speed_list = []
min_speed_list = []
for i in OOCL_GENOA_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in OOCL_GENOA_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 12.833333333333334
보간법 전 일반 속도 표준편차: 2.525015707588852
보간법 전 저속 제거 평균 속도 : 12.833333333333334
보간법 전 저속 제거 속도 표준편차: 2.525015707588852
-
보간법 후 일반 평균 속도 : 13.855634625333682
보간법 후 일반 속도 표준편차: 2.9298261480741052
보간법 후 저속 제거 평균 속도 : 13.855634625333682
보간법 후 저속 제거 속도 표준편차: 2.9298261480741052
보간법 전 일반 평균 속도 : 12.093772893772893
보간법 전 일반 속도 표준편차: 1.7522668237921277
보간법 전 저속 제거 평균 속도 : 12.093772893772893
보간법 전 저속 제거 속도 표준편차: 1.7522668237921277
-
보간법 후 일반 평균 속도 : 11.774121629529201
보간법 후 일반 속도 표준편차: 1.631708099228068
보간법 후 저속 제거 평균 속도 : 11.774121629529201
보간법 후 저속 제거 속도 표준편차: 1.631708099228068


In [17]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['OOCL_GENOA_VROK9_2022_3', 'OOCL_GENOA_VROK9_2022_5']

In [18]:
low_speed_path_data =[]
low_speed_path_data =[]
for i in OOCL_GENOA_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
    elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)

In [19]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [20]:
result_df_dict=  {
    "experiment_no" : [],
    "mean_speed" : [],
    "max_speed" : []
}

# ETA 실험 (interpolation 12000),  OOCL_GENOA 3항차

In [21]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.671903e+09,117.817600,38.966100,11.850000
1,1.671903e+09,117.819066,38.965853,11.919086
2,1.671903e+09,117.820531,38.965605,11.988172
3,1.671903e+09,117.821997,38.965358,12.057259
4,1.671903e+09,117.823462,38.965111,12.126345
...,...,...,...,...
11995,1.672110e+09,128.841559,34.928083,10.371201
11996,1.672110e+09,128.840994,34.928850,10.353401
11997,1.672110e+09,128.840429,34.929616,10.335601
11998,1.672110e+09,128.839865,34.930383,10.317800


In [22]:
the_number_of_segment = 120

## 저속 (7knot 미만 제거)

### segment 

In [23]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [24]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [25]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

12.524397830868558
12.038711705234709
10.729819781530196
12.8431123899921
12.65990909752217
12.778167335743472
12.535797681357387
12.46059662172548
12.729242393856307
12.8799584911063
12.940423468484337
12.979515231220187
13.000523911585628
13.006323130848925
13.012122350112206
13.017921569375638
13.023720788638933
12.79283178470522
12.659595204246273
12.705730755606776
12.749752515640232
12.73200749585475
12.871386373360819
12.797689519341594
12.750081511970976
12.666447451048679
12.689477057734223
12.736435294038982
12.747304783249934
12.707841489126144
12.698435188961666
12.749094899338031
13.007264706633231
13.738959492652993
14.349999999999993
14.223424647198936
14.202917674183556
14.209001938783231
14.215072405991304
14.221142873199376
14.227213340407445
14.233283807615512
14.239354274823599
14.245424742031657
14.270579793776925
14.24156735201912
14.262043164139104
14.400000000000007
14.354263575565469
14.274880017932308
14.3
14.3
14.3
14.299992860201572
14.297502815051793
14.293

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [26]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_GENOA_VROK9_2022_3processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [27]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_3".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [28]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:43:28.491932
평균: 0 days 02:05:25.538422060


In [29]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
205,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311100,128.3103,34.3152,9.5,34.0,2022-12-31 14:08:53.586028,2022-12-31 05:07:29,2022-12-31 05:07:52,under way using engine,86.396479,5.934970,2022-12-31 18:24:26.627438,2022-12-31 18:24:32.781137,0 days 00:00:06.153699
26,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.4752,38.5764,16.1,92.0,2022-12-29 21:55:27.544463,2022-12-29 12:53:15,2022-12-29 12:37:53,under way using engine,844.022223,73.771308,2022-12-31 18:24:22.232024,2022-12-31 18:24:32.781137,0 days 00:00:10.549113
211,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311100,128.3821,34.3618,9.3,57.0,2022-12-31 14:38:58.204127,2022-12-31 05:36:17,2022-12-31 05:07:52,under way using engine,78.533301,5.408943,2022-12-31 18:26:08.498490,2022-12-31 18:24:32.781137,0 days 00:01:35.717353
218,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311050,128.5204,34.4384,9.5,55.0,2022-12-31 15:34:06.147105,2022-12-31 06:28:56,2022-12-31 06:25:50,under way using engine,65.086148,4.471066,2022-12-31 18:22:35.570422,2022-12-31 18:24:32.781137,0 days 00:01:57.210715
226,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310945,128.6881,34.5380,15.2,49.0,2022-12-31 16:29:15.006156,2022-12-31 07:25:49,2022-12-31 07:28:00,under way using engine,49.469290,4.089932,2022-12-31 18:22:25.892491,2022-12-31 18:24:32.781137,0 days 00:02:06.888646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,124.7865,34.8169,14.8,151.0,2022-12-30 18:10:49.676711,2022-12-30 09:10:28,2022-12-29 03:49:54,under way using engine,368.189175,29.734494,2022-12-31 14:46:05.196995,2022-12-31 18:24:32.781137,0 days 03:38:27.584142
78,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,125.0045,34.4263,12.6,158.0,2022-12-30 20:06:07.846136,2022-12-30 11:05:38,2022-12-29 03:49:54,under way using engine,353.935013,25.638254,2022-12-31 14:45:21.919474,2022-12-31 18:24:32.781137,0 days 03:39:10.861663
72,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,124.9563,34.5224,12.4,158.0,2022-12-30 19:36:03.214993,2022-12-30 10:35:18,2022-12-29 03:49:54,under way using engine,356.428320,25.622025,2022-12-31 14:42:50.953379,2022-12-31 18:24:32.781137,0 days 03:41:41.827758
83,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,125.0444,34.3453,12.2,158.0,2022-12-30 20:31:11.491313,2022-12-30 11:30:38,2022-12-29 03:49:54,under way using engine,352.171570,25.134712,2022-12-31 14:42:23.649517,2022-12-31 18:24:32.781137,0 days 03:42:09.131620


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [30]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_GENOA_VROK9_2022_3processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [31]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_3".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [32]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:43:28.491932
평균: 0 days 02:00:36.978557778


In [33]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
205,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311100,128.3103,34.3152,9.5,34.0,2022-12-31 14:08:53.586028,2022-12-31 05:07:29,2022-12-31 05:07:52,under way using engine,86.396479,5.934970,2022-12-31 18:24:26.627438,2022-12-31 18:24:32.781137,0 days 00:00:06.153699
26,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.4752,38.5764,16.1,92.0,2022-12-29 21:55:27.544463,2022-12-29 12:53:15,2022-12-29 12:37:53,under way using engine,844.022223,73.771308,2022-12-31 18:24:22.232024,2022-12-31 18:24:32.781137,0 days 00:00:10.549113
211,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311100,128.3821,34.3618,9.3,57.0,2022-12-31 14:38:58.204127,2022-12-31 05:36:17,2022-12-31 05:07:52,under way using engine,78.533301,5.408943,2022-12-31 18:26:08.498490,2022-12-31 18:24:32.781137,0 days 00:01:35.717353
218,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12311050,128.5204,34.4384,9.5,55.0,2022-12-31 15:34:06.147105,2022-12-31 06:28:56,2022-12-31 06:25:50,under way using engine,65.086148,4.471066,2022-12-31 18:22:35.570422,2022-12-31 18:24:32.781137,0 days 00:01:57.210715
226,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310945,128.6881,34.5380,15.2,49.0,2022-12-31 16:29:15.006156,2022-12-31 07:25:49,2022-12-31 07:28:00,under way using engine,49.469290,4.089932,2022-12-31 18:22:25.892491,2022-12-31 18:24:32.781137,0 days 00:02:06.888646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,124.7865,34.8169,14.8,151.0,2022-12-30 18:10:49.676711,2022-12-30 09:10:28,2022-12-29 03:49:54,under way using engine,368.189175,29.734494,2022-12-31 14:46:05.196995,2022-12-31 18:24:32.781137,0 days 03:38:27.584142
78,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,125.0045,34.4263,12.6,158.0,2022-12-30 20:06:07.846136,2022-12-30 11:05:38,2022-12-29 03:49:54,under way using engine,353.935013,25.638254,2022-12-31 14:45:21.919474,2022-12-31 18:24:32.781137,0 days 03:39:10.861663
72,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,124.9563,34.5224,12.4,158.0,2022-12-30 19:36:03.214993,2022-12-30 10:35:18,2022-12-29 03:49:54,under way using engine,356.428320,25.622025,2022-12-31 14:42:50.953379,2022-12-31 18:24:32.781137,0 days 03:41:41.827758
83,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,125.0444,34.3453,12.2,158.0,2022-12-30 20:31:11.491313,2022-12-30 11:30:38,2022-12-29 03:49:54,under way using engine,352.171570,25.134712,2022-12-31 14:42:23.649517,2022-12-31 18:24:32.781137,0 days 03:42:09.131620


# ETA 실험 (interpolation 12000),  OOCL_GENOA 5항차

In [34]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.671903e+09,117.817600,38.966100,11.850000
1,1.671903e+09,117.819066,38.965853,11.919086
2,1.671903e+09,117.820531,38.965605,11.988172
3,1.671903e+09,117.821997,38.965358,12.057259
4,1.671903e+09,117.823462,38.965111,12.126345
...,...,...,...,...
11995,1.672110e+09,128.841559,34.928083,10.371201
11996,1.672110e+09,128.840994,34.928850,10.353401
11997,1.672110e+09,128.840429,34.929616,10.335601
11998,1.672110e+09,128.839865,34.930383,10.317800


## 저속 (7knot 미만 제거)

### segment 

In [35]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [36]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [37]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

12.524397830868558
12.038711705234709
10.729819781530196
12.8431123899921
12.65990909752217
12.778167335743472
12.535797681357387
12.46059662172548
12.729242393856307
12.8799584911063
12.940423468484337
12.979515231220187
13.000523911585628
13.006323130848925
13.012122350112206
13.017921569375638
13.023720788638933
12.79283178470522
12.659595204246273
12.705730755606776
12.749752515640232
12.73200749585475
12.871386373360819
12.797689519341594
12.750081511970976
12.666447451048679
12.689477057734223
12.736435294038982
12.747304783249934
12.707841489126144
12.698435188961666
12.749094899338031
13.007264706633231
13.738959492652993
14.349999999999993
14.223424647198936
14.202917674183556
14.209001938783231
14.215072405991304
14.221142873199376
14.227213340407445
14.233283807615512
14.239354274823599
14.245424742031657
14.270579793776925
14.24156735201912
14.262043164139104
14.400000000000007
14.354263575565469
14.274880017932308
14.3
14.3
14.3
14.299992860201572
14.297502815051793
14.293

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [38]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_GENOA_VROK9_2022_5processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [39]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_5".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [40]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 07:42:27.015625
평균: 0 days 01:47:55.402285912


In [41]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
191,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,127.3951,33.8522,13.6,81.0,2022-12-22 02:59:54.098072,2022-12-21 17:59:35,2022-12-21 03:44:15,under way using engine,180.304403,18.746397,2022-12-22 11:31:29.464772,2022-12-22 11:31:12.768951,0 days 00:00:16.695821
227,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,128.0689,34.1974,13.7,63.0,2022-12-22 06:00:21.843742,2022-12-21 20:59:21,2022-12-21 03:44:15,under way using engine,109.971412,11.473793,2022-12-22 11:31:56.266534,2022-12-22 11:31:12.768951,0 days 00:00:43.497583
130,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,125.0091,34.5313,12.1,159.0,2022-12-21 16:23:17.301683,2022-12-21 07:22:32,2022-12-20 11:56:16,under way using engine,351.495767,35.130989,2022-12-22 11:30:05.040069,2022-12-22 11:31:12.768951,0 days 00:01:07.728882
222,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,127.9643,34.1609,14.0,66.0,2022-12-22 05:35:18.109810,2022-12-21 20:34:58,2022-12-21 03:44:15,under way using engine,119.236680,12.578404,2022-12-22 11:30:00.072915,2022-12-22 11:31:12.768951,0 days 00:01:12.696036
140,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,125.1096,34.3542,12.6,153.0,2022-12-21 17:18:25.491634,2022-12-21 08:18:02,2022-12-20 11:56:16,under way using engine,346.113348,34.949383,2022-12-22 11:29:37.649838,2022-12-22 11:31:12.768951,0 days 00:01:35.119113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,118.0828,38.9152,9.5,115.0,2022-12-19 22:36:54.799404,2022-12-19 13:32:52,2022-12-19 01:08:15,under way using engine,1049.292816,105.798597,2022-12-22 04:03:51.775097,2022-12-22 11:31:12.768951,0 days 07:27:20.993854
2,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.8845,38.9556,12.1,98.0,2022-12-19 21:41:44.958667,2022-12-19 12:37:57,2022-12-19 12:32:17,under way using engine,1066.905668,106.634146,2022-12-22 04:03:48.113170,2022-12-22 11:31:12.768951,0 days 07:27:24.655781
4,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.9549,38.9468,13.3,102.0,2022-12-19 21:56:46.998770,2022-12-19 12:53:37,2022-12-19 12:32:17,under way using engine,1060.876311,109.215401,2022-12-22 03:54:53.243495,2022-12-22 11:31:12.768951,0 days 07:36:19.525456
1,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.8447,38.9611,13.2,100.0,2022-12-19 21:31:43.965116,2022-12-19 12:29:52,2022-12-19 11:32:13,under way using engine,1070.336129,109.849178,2022-12-22 03:53:47.119619,2022-12-22 11:31:12.768951,0 days 07:37:25.649332


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [42]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_GENOA_VROK9_2022_5processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [43]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_5".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [44]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 07:42:27.015625
평균: 0 days 01:49:18.943444278


In [45]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
191,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,127.3951,33.8522,13.6,81.0,2022-12-22 02:59:54.098072,2022-12-21 17:59:35,2022-12-21 03:44:15,under way using engine,180.304403,18.746397,2022-12-22 11:31:29.464772,2022-12-22 11:31:12.768951,0 days 00:00:16.695821
227,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,128.0689,34.1974,13.7,63.0,2022-12-22 06:00:21.843742,2022-12-21 20:59:21,2022-12-21 03:44:15,under way using engine,109.971412,11.473793,2022-12-22 11:31:56.266534,2022-12-22 11:31:12.768951,0 days 00:00:43.497583
130,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,125.0091,34.5313,12.1,159.0,2022-12-21 16:23:17.301683,2022-12-21 07:22:32,2022-12-20 11:56:16,under way using engine,351.495767,35.130989,2022-12-22 11:30:05.040069,2022-12-22 11:31:12.768951,0 days 00:01:07.728882
222,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220300,127.9643,34.1609,14.0,66.0,2022-12-22 05:35:18.109810,2022-12-21 20:34:58,2022-12-21 03:44:15,under way using engine,119.236680,12.578404,2022-12-22 11:30:00.072915,2022-12-22 11:31:12.768951,0 days 00:01:12.696036
140,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,125.1096,34.3542,12.6,153.0,2022-12-21 17:18:25.491634,2022-12-21 08:18:02,2022-12-20 11:56:16,under way using engine,346.113348,34.949383,2022-12-22 11:29:37.649838,2022-12-22 11:31:12.768951,0 days 00:01:35.119113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.8845,38.9556,12.1,98.0,2022-12-19 21:41:44.958667,2022-12-19 12:37:57,2022-12-19 12:32:17,under way using engine,1066.905668,106.634146,2022-12-22 04:03:48.113170,2022-12-22 11:31:12.768951,0 days 07:27:24.655781
4,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.9549,38.9468,13.3,102.0,2022-12-19 21:56:46.998770,2022-12-19 12:53:37,2022-12-19 12:32:17,under way using engine,1060.876311,109.215401,2022-12-22 03:54:53.243495,2022-12-22 11:31:12.768951,0 days 07:36:19.525456
7,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,118.0600,38.9242,6.6,118.0,2022-12-19 22:26:51.560222,2022-12-19 13:23:58,2022-12-19 01:08:15,under way using engine,1051.494428,127.628908,2022-12-22 03:53:48.535915,2022-12-22 11:31:12.768951,0 days 07:37:24.233036
1,OOCL GENOA,477454900,9613587.0,8825,KRBNP,12220400,117.8447,38.9611,13.2,100.0,2022-12-19 21:31:43.965116,2022-12-19 12:29:52,2022-12-19 11:32:13,under way using engine,1070.336129,109.849178,2022-12-22 03:53:47.119619,2022-12-22 11:31:12.768951,0 days 07:37:25.649332


# 결과 확인

In [46]:
result_pool = pd.DataFrame(result_df_dict)
result_pool

,experiment_no,mean_speed,max_speed
0,4_1_2_3_seg_120_voy_3,0 days 02:05:25.538422060,0 days 03:43:28.491932
1,4_2_1_3_seg_120_voy_3,0 days 02:00:36.978557778,0 days 03:43:28.491932
2,4_1_2_3_seg_120_voy_5,0 days 01:47:55.402285912,0 days 07:42:27.015625
3,4_2_1_3_seg_120_voy_5,0 days 01:49:18.943444278,0 days 07:42:27.015625


In [47]:
result_pool.sort_values(by="mean_speed")

,experiment_no,mean_speed,max_speed
2,4_1_2_3_seg_120_voy_5,0 days 01:47:55.402285912,0 days 07:42:27.015625
3,4_2_1_3_seg_120_voy_5,0 days 01:49:18.943444278,0 days 07:42:27.015625
1,4_2_1_3_seg_120_voy_3,0 days 02:00:36.978557778,0 days 03:43:28.491932
0,4_1_2_3_seg_120_voy_3,0 days 02:05:25.538422060,0 days 03:43:28.491932


In [48]:
for i in globals():
    if 'exp_result_df_' in i:
        globals()[i].to_csv(f"실험/230206_{vessel_name}_{i}", encoding="utf-8 sig", header=True, index=False)

In [49]:
# exp_result_df_4_1_2_3

In [50]:
# result_pool.to_csv(f"230126_result_pool_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)